In [ ]:
#! python --version

In [ ]:
#! pip3 install gym

In [ ]:
#! pip3 install pygame

# Create the environment

In [3]:
import gym

In [2]:
env = gym.make("CartPole-v1", render_mode="human")

In [4]:
env.reset()

(array([-0.04511656,  0.00330043,  0.01300843,  0.00355254], dtype=float32),
 {})

In [ ]:
# neutral state

#env.reset()

In [5]:
env.close()

In [6]:
env.action_space

Discrete(2)

In [12]:
env.action_space.sample()

1

In [18]:
# this shows the cartpole in the environment when we take random actions

env = gym.make("CartPole-v1", render_mode="human")

episodes = 50

for e in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample() # we are taking random actions, left and right
        n_state, reward, done, _, info = env.step(action)
        score += reward
    print(f"Episode #: {e}")
    print(f"Score: {score}")

env.close()

Episode #: 1
Score: 20.0
Episode #: 2
Score: 40.0
Episode #: 3
Score: 23.0
Episode #: 4
Score: 20.0
Episode #: 5
Score: 16.0
Episode #: 6
Score: 17.0
Episode #: 7
Score: 16.0
Episode #: 8
Score: 19.0
Episode #: 9
Score: 16.0
Episode #: 10
Score: 20.0
Episode #: 11
Score: 24.0
Episode #: 12
Score: 24.0
Episode #: 13
Score: 18.0
Episode #: 14
Score: 12.0
Episode #: 15
Score: 18.0
Episode #: 16
Score: 9.0
Episode #: 17
Score: 21.0
Episode #: 18
Score: 14.0
Episode #: 19
Score: 32.0
Episode #: 20
Score: 11.0
Episode #: 21
Score: 12.0
Episode #: 22
Score: 29.0
Episode #: 23
Score: 19.0
Episode #: 24
Score: 16.0
Episode #: 25
Score: 24.0
Episode #: 26
Score: 14.0
Episode #: 27
Score: 17.0
Episode #: 28
Score: 15.0
Episode #: 29
Score: 49.0
Episode #: 30
Score: 36.0
Episode #: 31
Score: 20.0
Episode #: 32
Score: 13.0
Episode #: 33
Score: 39.0


KeyboardInterrupt: 

In [ ]:
env.close()

## Train the agent

In [ ]:
!pip3 install 'stable-baselines3[extra]'

In [28]:
from stable_baselines3 import PPO # the algorithm we are importing
# reference: https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html
from stable_baselines3.common.vec_env import DummyVecEnv
# vectorise environment - train on multiple agents
# wrapper around environment
from stable_baselines3.common.evaluation import evaluate_policy

In [29]:
PPO?

Init signature:
PPO(
    policy: Union[str, Type[stable_baselines3.common.policies.ActorCriticPolicy]],
    env: Union[gymnasium.core.Env, ForwardRef('VecEnv'), str],
    learning_rate: Union[float, Callable[[float], float]] = 0.0003,
    n_steps: int = 2048,
    batch_size: int = 64,
    n_epochs: int = 10,
    gamma: float = 0.99,
    gae_lambda: float = 0.95,
    clip_range: Union[float, Callable[[float], float]] = 0.2,
    clip_range_vf: Union[NoneType, float, Callable[[float], float]] = None,
    normalize_advantage: bool = True,
    ent_coef: float = 0.0,
    vf_coef: float = 0.5,
    max_grad_norm: float = 0.5,
    use_sde: bool = False,
    sde_sample_freq: int = -1,
    rollout_buffer_class: Optional[Type[stable_baselines3.common.buffers.RolloutBuffer]] = None,
    rollout_buffer_kwargs: Optional[Dict[str, Any]] = None,
    target_kl: Optional[float] = None,
    stats_window_size: int = 100,
    tensorboard_log: Optional[str] = None,
    policy_kwargs: Optional[Dict[str, Any]]

In [50]:
# create our environment again
env = gym.make("CartPole-v1", render_mode="human")
# vectorise our environment
env = DummyVecEnv([lambda:env])
# import the algorithm
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log="training/logs")

Using cpu device


In [51]:
model.learn(total_timesteps=20000)

Logging to training/logs/PPO_8
-----------------------------
| time/              |      |
|    fps             | 46   |
|    iterations      | 1    |
|    time_elapsed    | 44   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 46          |
|    iterations           | 2           |
|    time_elapsed         | 88          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009046177 |
|    clip_fraction        | 0.113       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.0031      |
|    learning_rate        | 0.0003      |
|    loss                 | 5.09        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0166     |
|    value_loss           | 45.4        |
-----------------------------------------
---

In [52]:
model_path = "training/models/cartpole_20000"
model.save(model_path)

In [53]:
del model

## Evaluate the Model

In [54]:
# import model
env = gym.make("CartPole-v1", render_mode="human")
model_path = "training/models/cartpole_20000"
model = PPO.load(model_path, env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [55]:
evaluate_policy(model, env, n_eval_episodes=5, render=True)

(500.0, 0.0)

In [56]:
env.close()

In [57]:
env.action_space.sample()

1

## Test the Model

In [59]:
# test the model
env = gym.make("CartPole-v1", render_mode="human")
model_path = "training/models/cartpole_20000"
model = PPO.load(model_path, env=env)

episodes = 5

for e in range(1, episodes+1):
    obs, _ = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, _, info = env.step(action)
        score += reward
    print(f"Episode #: {e}")
    print(f"Score: {score}")

env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Episode #: 1
Score: 525.0
Episode #: 2
Score: 911.0
Episode #: 3
Score: 1103.0
Episode #: 4
Score: 511.0
Episode #: 5
Score: 453.0


In [1]:
! pip3 install tensorboard

In [7]:
!pip install tensorflow

  Using cached tensorflow-2.16.1-cp311-cp311-macosx_10_15_x86_64.whl.metadata (4.1 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.5.4-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.11.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-macosx_10_9_x86_64.whl.metadata (5.2 kB)
  Using cached ml_dtypes-0.3.2-cp311-cp311-macosx_10_9_universal2.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.16.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata (6.6 kB)
  Using cached keras-3.3.3-py3-none-any.whl.metadata (5.7 kB)
  Using cached 

In [ ]:
log_path = 'training/logs/PPO_8'
!tensorboard --logdir={log_path}